In [ ]:
import pandas as pd
import requests
import altair as alt

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/EconomicsObservatory/courses/HEAD?labpath=5%2Fs5_transforming_data.ipynb)

<a href="https://colab.research.google.com/github/EconomicsObservatory/courses/blob/main/5/s5_transforming_data.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# A Transforming Data

The bread and butter of a data workflow is cleaning and preparation, taking raw datasets and transforming them into a useful form.

Today we're going to produce a chart using two series from the Economics Observatory API that we'll transform with `Pandas` and present with `Vega-lite`. We will produce a chart showing Indian Exports and Imports as a percentage of GDP using data sourced from the API.

</br></br></br></br>

</br></br></br></br>


</br></br></br></br>



## Introducing Tools: Pandas

The first tool we'll use today is `Pandas`, a Python library used to work with datasets. It provides access to `DataFrames` - tables we analyse with code.

Python already has a few built in data structures, for example lists and dictionaries:

</br></br></br>

</br>
</br>
</br>
</br></br></br>
</br></br>


# Introducing Tools: `Requests`

The `Requests` module allows us to fetch resources from the internet, whether these are `CSVs`, `JSONs`, images, `HTML` or anything else. This is particuarly important for requesting data from APIs.

</br>

It is simple to use. Usually allow we need to do is:

1. Make a request with `requests.get` and our target URL. For example we can request something from our GitHub repos:

    `req = requests.get("https://raw.githubusercontent.com/mclass-user/mclass-user.github.io/main/s2_chart1.json")`
</br>
</br>
2. Access the fetched data. Using `req.json()` for JSON data or `req.text` for most other data. For example, we can see the returned JSON for the chart we just fetched:
    </br>
    </br>
    `data = req.json()`

In [ ]:
req = requests.get("https://raw.githubusercontent.com/mclass-user/mclass-user.github.io/main/s2_chart1.json")


data = req.json()
data

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json',
 'title': {'text': 'Human Development Index ',
  'subtitle': ["P21 Countries' HDI, most recent year", 'Source: UN']},
 'description': 'A simple bar chart with embedded data.',
 'data': {'values': [{'Country': 'Bangladesh', 'HDI': 0.661},
   {'Country': 'Brazil', 'HDI': 0.754},
   {'Country': 'China', 'HDI': 0.768},
   {'Country': 'DR Congo', 'HDI': 0.479},
   {'Country': 'Egypt', 'HDI': 0.731},
   {'Country': 'Ethiopia', 'HDI': 0.498},
   {'Country': 'Germany', 'HDI': 0.942},
   {'Country': 'India', 'HDI': 0.633},
   {'Country': 'Indonesia', 'HDI': 0.705},
   {'Country': 'Iran', 'HDI': 0.774},
   {'Country': 'Japan', 'HDI': 0.925},
   {'Country': 'Mexico', 'HDI': 0.758},
   {'Country': 'Nigeria', 'HDI': 0.535},
   {'Country': 'Pakistan', 'HDI': 0.544},
   {'Country': 'Philippines', 'HDI': 0.699},
   {'Country': 'Russia', 'HDI': 0.829},
   {'Country': 'Thailand', 'HDI': 0.8},
   {'Country': 'Turkey', 'HDI': 0.838},
   {'Coun

</br></br>
</br></br>
</br>
</br>
</br>

Transforming the data

We need to produce a chart showing UK inflation rates and unemployment rates. To do this, we can use two series from the [Economics Observatory API](https://www.economicsobservatory.com/data-hub):
1. UK inflation rate
2. UK unemployment rate


</br>
</br>

We can find the API urls for these from the [Data-Hub](https://www.economicsobservatory.com/data-hub)


</br>
</br>

We find:

1. **Inflation**: https://api.economicsobservatory.com/gbr/infl
2. **Unemployment rate**: https://api.economicsobservatory.com/gbr/unemp


</br>
</br>


Let's try requesting the first of these, `Inflation rate` and see how it's formatted.

In [ ]:
inflation_req = requests.get("https://api.economicsobservatory.com/gbr/infl")
inflation_req.json()

{'author': 'Economics Observatory',
 'source': 'ONS',
 'url': 'https://api.ons.gov.uk/timeseries/L55O/dataset/MM23/data',
 'data': [{'date': '1989-01', 'value': 5.7},
  {'date': '1989-02', 'value': 5.8},
  {'date': '1989-03', 'value': 5.9},
  {'date': '1989-04', 'value': 5.6},
  {'date': '1989-05', 'value': 5.9},
  {'date': '1989-06', 'value': 5.8},
  {'date': '1989-07', 'value': 5.7},
  {'date': '1989-08', 'value': 5.5},
  {'date': '1989-09', 'value': 5.7},
  {'date': '1989-10', 'value': 5.8},
  {'date': '1989-11', 'value': 5.7},
  {'date': '1989-12', 'value': 5.8},
  {'date': '1990-01', 'value': 5.9},
  {'date': '1990-02', 'value': 6.3},
  {'date': '1990-03', 'value': 6.2},
  {'date': '1990-04', 'value': 7.7},
  {'date': '1990-05', 'value': 8.2},
  {'date': '1990-06', 'value': 8.2},
  {'date': '1990-07', 'value': 8.2},
  {'date': '1990-08', 'value': 8.9},
  {'date': '1990-09', 'value': 9.2},
  {'date': '1990-10', 'value': 9.2},
  {'date': '1990-11', 'value': 9.2},
  {'date': '1990-12

</br>
</br>

We just care about the data itself. Let's take this and make a dataframe out of it.

We'll rename the `value` column to `inflation` so we can keep track of what this data is when we merge it.

In [ ]:
inflation_response = inflation_req.json()
inflation_data = inflation_response['data']
inflation_df = pd.DataFrame(inflation_data)
inflation_df = inflation_df.rename(columns={"value": "inflationrate"})
inflation_df

,date,inflationrate
0,1989-01,5.7
1,1989-02,5.8
2,1989-03,5.9
3,1989-04,5.6
4,1989-05,5.9
...,...,...
424,2024-05,2.8
425,2024-06,2.8
426,2024-07,3.1
427,2024-08,3.1


</br>
</br>
</br>

We now have a Pandas `DataFrame` containing the inflation data. Let's do the same for unemplyoyment data.

In [ ]:
unemployment_url = "https://api.economicsobservatory.com/gbr/unem"
unemployment_req = requests.get(unemployment_url) # This is the request
unemployment_response = unemployment_req.json() # This is the response in JSON format
unemployment_data = unemployment_response['data'] # We only want the data part of the response, not the metadata
unemployment_df = pd.DataFrame(unemployment_data) # We convert the data to a DataFrame
unemployment_df = unemployment_df.rename(columns={"value": "unemploymentrate"}) # We rename the value column to unemployment


</br>
</br>

Now that we have all three series downloaded, we can start merging and transforming the data.

Let's merge our `gdp_df` with the `exports_df` and `imports_df` so we can express them as a ratio.

In [ ]:
inflation_ratio_df = inflation_df.merge(unemployment_df, on="date")
inflation_ratio_df.tail(5)

,date,inflationrate,unemploymentrate
423,2024-04,3.0,4.4
424,2024-05,2.8,4.2
425,2024-06,2.8,4.1
426,2024-07,3.1,4.0
427,2024-08,3.1,4.3


</br>
</br>

Let's now divide the `exports` column by the `gdp` column to get the Ratio and keep only the result and the `date`.

<br/><br/>

We now have all the data we need in two dataframes: `inflation_ratio_df` and `unemployment_ratio_df`.

To use these in a graph, we'll need them in one dataframe.

In [ ]:
economy_df = pd.merge(inflation_df, unemployment_df, on="date") # Merge the inflation rate and unemployment rate DataFrames on the date column
economy_df.head(5)

,date,inflationrate,unemploymentrate
0,1989-01,5.7,7.7
1,1989-02,5.8,7.6
2,1989-03,5.9,7.4
3,1989-04,5.6,7.3
4,1989-05,5.9,7.2


</br></br>

It's best practice for `Vega-lite` to keep data in a TIDY format where there are columns that specify the series and a column for the data. To tranform our data, we'll use the pandad function `melt`.

In [ ]:
economy_df = economy_df.melt(id_vars="date", var_name="series", value_name="value") # Melt the DataFrame so that the inflation rate and unemployment rates are in the same column,
                                                                                # Specifying we want columns of date and series
economy_df

,date,series,value
0,1989-01,inflationrate,5.7
1,1989-02,inflationrate,5.8
2,1989-03,inflationrate,5.9
3,1989-04,inflationrate,5.6
4,1989-05,inflationrate,5.9
...,...,...,...
851,2024-04,unemploymentrate,4.4
852,2024-05,unemploymentrate,4.2
853,2024-06,unemploymentrate,4.1
854,2024-07,unemploymentrate,4.0


</br>
</br>

We can now export this data to use in `Vega-lite`.

In [ ]:
economy_df.to_csv("s5_UK_economy_data.csv", index=False)

</br></br>

Finally, to get the CSV onto GitHub from Colab. We just have to go to the Files tab in the sidebar, and find our file:

<img
style="max-height: 350px;
    width: auto;" src="https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/misc/Masterclass/section%205/images/colab_download.png">
</img>